In [1]:
%load_ext autotime

In [2]:
import sys
sys.path.append('/Users/Pradap/Documents/Research/Python-Package/enrique/')

time: 965 µs


In [3]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 540 ms


In [4]:
mg.init_jvm('/Library/Java/JavaVirtualMachines/jdk1.8.0_45.jdk/Contents/Home/jre/lib/server/libjvm.dylib')

True

time: 101 ms


In [5]:
# read walmart dataset
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv', 
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.21 s


In [6]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')

time: 85.4 ms


In [7]:
wal.columns

Index([u'id', u'title', u'author', u'numAuthors', u'binding', u'publisher',
       u'isbn', u'pubYear', u'pubMonth', u'pubDay', u'editionNum', u'pages',
       u'volume', u'lang', u'editionDescr', u'category', u'upc11', u'upc12',
       u'upc13', u'upc14', u'description', u'url'],
      dtype='object')

time: 2.47 ms


In [8]:
bwk.columns

Index([u'id', u'title', u'author', u'numAuthors', u'binding', u'publisher',
       u'isbn', u'pubYear', u'pubMonth', u'pubDay', u'editionNum', u'pages',
       u'volume', u'lang', u'editionDescr', u'series', u'upc'],
      dtype='object')

time: 14.6 ms


In [9]:
ob = mg.OverlapBlocker()

time: 11.1 ms


In [10]:
mg._verbose=True
mg._percent=1

time: 1.36 ms


In [11]:
C = ob.block_tables(wal, bwk, 'binding', 'binding', overlap_size=2)

Creating inverted index 
Done
100.0
1.0 percentage done !!!
2.0 percentage done !!!
3.0 percentage done !!!
4.0 percentage done !!!
5.0 percentage done !!!
6.0 percentage done !!!
7.0 percentage done !!!
8.0 percentage done !!!
9.0 percentage done !!!
10.0 percentage done !!!
11.0 percentage done !!!
12.0 percentage done !!!
13.0 percentage done !!!
14.0 percentage done !!!
15.0 percentage done !!!
16.0 percentage done !!!
17.0 percentage done !!!
18.0 percentage done !!!
19.0 percentage done !!!
20.0 percentage done !!!
21.0 percentage done !!!
22.0 percentage done !!!
23.0 percentage done !!!
24.0 percentage done !!!
25.0 percentage done !!!
26.0 percentage done !!!
27.0 percentage done !!!
28.0 percentage done !!!
29.0 percentage done !!!
30.0 percentage done !!!
31.0 percentage done !!!
32.0 percentage done !!!
33.0 percentage done !!!
34.0 percentage done !!!
35.0 percentage done !!!
36.0 percentage done !!!
37.0 percentage done !!!
38.0 percentage done !!!
39.0 percentage done !!

/Users/Pradap/Documents/Research/Python-Package/enrique/magellan/blocker/overlap_blocker.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  l_rows_dict['dummy'] = 1
